### Import semua library yang dibutuhkan

In [83]:
import sys
import os
sys.path.append("../../")
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import modules.ReadCsv as rc
import modules.PreProcess as pp
import model.ListOfModels as lm

from subproc.Vocabulary import *
from keras.preprocessing import sequence
from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import keras_metrics as km

import numpy as np
import json

### Deklarasi konstanta

In [84]:
NUM_OF_ATTRIBUTES = 2500
vocabPath = '../../../vocabulary/corpus.json'
inputPath = '../../../input/data_testing_225.csv'
slangWordPath = '../../../vocabulary/slangwords.csv'

### Proses membaca data testing dan melakukan preprocess

In [85]:
posData, negData = rc.csv2array(inputPath)
posData = pp.getResult(posData, slangWordPath)
negData = pp.getResult(negData, slangWordPath)

### Tahap memberi label pada data

In [86]:
dataLabeled = list(zip(posData, np.ones(len(posData))))
dataLabeled.extend(list(zip(negData, np.zeros(len(negData)))))

for i in dataLabeled:
    print(i)

('fadli zon mendagri menonaktifkan ahok gubernur dki', 1.0)
('melukai aksi rangka memenjarakan ahok ahok gagal pilkada', 1.0)
('sylvi gurbernur kekerasan perempuan buktinya foto bareng', 1.0)
('ahmad dhani puas debat pilkada jalan bekasi terungkap', 1.0)
('waspada ktp palsu kawal pilkada', 1.0)
('bnyak agamis dibahas pilkada tingkat keimanan sebatas pilkada', 1.0)
('maksud pernyataan mengaminkan kriminalisasi ahok hati dg mulutmu', 1.0)
('ahok integrasikan transportasi publik sistem single ticketing', 1.0)
('ahok melawan korupsi ibukota bangun infrastruktur mencoba memperbaiki trobosan works well', 1.0)
('awasi kecurangan pilkada agus sylvi pakai aplikasi', 1.0)
('jokowi ahok kalem melengkapi', 1.0)
('bos makan istana undang noh makan istana gebang tapinya', 1.0)
('cuti habis ahok gubernur dki pengamat mengada republika online', 1.0)
('debat pilkada dki jakarta digelar jumat malam', 1.0)
('demonstran fpd malteng desak bawaslu hentikan tahapan pilkada maluku', 1.0)
('didesak berhentikan

### Proses merubah label dalam bentuk one hot

In [87]:
datas, labels = zip(*dataLabeled) # proses memisahkan data dengan label
labels = to_categorical(labels) # proses merubah label dalam bentuk one hot

### Merubah data menjadi angka dan melakukan training

In [88]:
vocab = Vocabulary()
x = vocab.transformSentencesToId2(datas, NUM_OF_ATTRIBUTES, vocabPath)
x = sequence.pad_sequences(x, maxlen=21)

model = load_model('../model/bi-lstm3.h5')
y = model.predict(np.array(x), verbose=0)
y = np.exp(y)
yPredict = []

for label in y:
    if label[0] < label[1]:
        yPredict.append(1.0)
    else:
        yPredict.append(0.0)
yPredict = to_categorical(yPredict)

### Perhitungan tp, tn, fp, fn

In [91]:
tp = 0
tn = 0
fp = 0
fn = 0

posLabel = np.array([0.0, 1.0])
negLabel = np.array([1.0, 0.0])

indexFp = 0
indexFn = 0

for index, (yVal, yPred) in enumerate(zip(labels, yPredict)):
    if (np.array_equal(yVal, posLabel)):
        if(np.array_equal(yVal, yPred)):
            tp += 1
        else:
            fp += 1
#             print('Index False Positive : ', index)
    else:
        if(np.array_equal(yVal, yPred)):
            tn += 1
        else:
            fn += 1
#             print('Index False Negative : ', index)

print('True Positif : ', tp)
print('True Negatif : ', tn)
print('False Positif : ', fp)
print('False Negatif : ', fn)

True Positif :  104
True Negatif :  101
False Positif :  7
False Negatif :  13


### Perhitungan Akurasi, presisi, dan recall

In [92]:
acc = (tp + tp) / (tp + tn + fp + fn)
prec = (tp) / (tp + fp)
rec = (tp) / (tp + fn)
f1 = ((prec * rec) / (prec + rec)) * 2

print('Akurasi : ', acc * 100)
print('Presisi : ', prec * 100)
print('Recall : ', rec * 100)
print('F1 : ', f1 * 100)

Akurasi :  92.44444444444444
Presisi :  93.69369369369369
Recall :  88.88888888888889
F1 :  91.22807017543859


## Contoh data yang salah

### False Positif
- Sylvie sukses blunder bikin paslon 1 drop elektabilitasnya dengan fitnah dan spin salah pilih kader #DebatFinalPilkadaJKT
- Spanduk Tolak Salatkan Pembela Penista Agama juga Muncul di Kalibata

### False Negatif
- Sayang sekali Bangsa ini Paling jago kalau merasa-rasa. Yg mendukung Pak Ahok merasa paling benar, yg mendukung Mas Anies merasa paling suci
- Mbak Sylviana please Anda sehat?